In [1]:
import os
import sys
import yaml
import json
import pandas as pd
import numpy as np
import datetime
import json
import yaml
import urllib3
from itertools import chain

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

sys.path.insert(0, os.path.abspath('../functions'))
from jira_api import get_all_tasks
from jira_api import flatten_dict

In [2]:
# load JIRA username and password
jira_creds = yaml.load(open('../user_dev.yml'))
username = jira_creds['user']['name']
password = jira_creds['user']['password']

In [3]:
# point to JIRA development environment
url = 'https://10.221.100.4'

## `../functions/get_all_task_status.py`

In [4]:
all_tasks = get_all_tasks(url,
                          username,
                          password,
                          "AI",
                          start_at=0,
                          max_results=100)

# pipe issues to txt
now = datetime.datetime.now()
today = now.strftime("%Y-%m-%d")

with open(
        '../status_report/raw/%s.txt' % str(today), 'w') as outfile:
    json.dump(all_tasks, outfile)


df = pd.DataFrame.from_dict(all_tasks['issues'])

# flatten all issues
explode_issues = pd.DataFrame.from_dict(
    list(map(lambda x: flatten_dict(x), df['fields'])))

field_mapping = json.load(open('../scripts/chc_jira_prod_fields.json'))

# find and replace customfields with field names
for i in np.arange(0, len(field_mapping)):
    explode_issues.columns = explode_issues.columns.str.replace(
                                            dict(field_mapping[i])['id'],
                                            dict(field_mapping[i])['name'])

# format column names to lower case w/o spaces
explode_issues.columns = explode_issues.columns.str.lower().str.replace(' ', '_')

# persist flattened issues to .csv
explode_issues.reset_index().to_csv(
                      '../status_report/flattened/tasks/%s.txt' % str(today),
                      index=False)

### kludgey update to extract and flatten sub-tasks

def flatten(l): 
    return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]

def coalesce_dicts(list_of_dicts):
    if len(list_of_dicts) == 1:
        return flatten(list_of_dicts[0])
    if len(list_of_dicts) > 1:
        dict1 = list_of_dicts[0]
        for i in np.arange(1, len(list_of_dicts)):
            dict1 = dict((k, flatten([dict1[k], list_of_dicts[i].get(k)])) for k in dict1)
    return dict1

def explode_json(d):
    if d == []:
        return np.nan
    else:
        return list(map(lambda x: flatten_dict(x), d))
    
def combine_json(l):
    if l is np.nan:
        return np.nan
    else:
        return pd.DataFrame.from_dict(coalesce_dicts(l))
    
    
temp = list(map(lambda x: explode_json(x), explode_issues['sub-tasks']))

temp_2 = pd.DataFrame(list(map(lambda x: combine_json(x), temp))).dropna()


indices = []
dfs = []

for i in temp_2.index:
    indices.append(i)
    dfs.append(temp_2.loc[i, 0])
    
temp_3 = pd.concat(dfs, axis=0, keys=indices)
column_names = 'subtask_' + temp_3.columns

temp_3.columns = column_names

temp_3 = temp_3.reset_index().drop('level_1', axis=1)

temp_3.columns = temp_3.columns.str.replace('level_0', 'index')

# persist flattened issues to .csv
temp_3.to_csv(
    '../status_report/flattened/sub_tasks/%s.txt' % str(today),
    index=False)

 |##############################################################################################------| 94.8% 

In [5]:
df['fields'][0]

{'aggregateprogress': {'progress': 0, 'total': 0},
 'aggregatetimeestimate': None,
 'aggregatetimeoriginalestimate': None,
 'aggregatetimespent': None,
 'assignee': None,
 'components': [],
 'created': '2017-11-16T22:55:33.000+0000',
 'creator': {'active': True,
  'avatarUrls': {'16x16': 'http://10.221.100.4/secure/useravatar?size=xsmall&ownerId=smacrae&avatarId=13320',
   '24x24': 'http://10.221.100.4/secure/useravatar?size=small&ownerId=smacrae&avatarId=13320',
   '32x32': 'http://10.221.100.4/secure/useravatar?size=medium&ownerId=smacrae&avatarId=13320',
   '48x48': 'http://10.221.100.4/secure/useravatar?ownerId=smacrae&avatarId=13320'},
  'displayName': 'Macrae, Sean',
  'emailAddress': 'sean.macrae@changehealthcare.com',
  'key': 'smacrae',
  'name': 'smacrae',
  'self': 'http://10.221.100.4/rest/api/2/user?username=smacrae',
  'timeZone': 'America/Chicago'},
 'customfield_10000': None,
 'customfield_10001': 'AI-1214',
 'customfield_10002': '9223372036854775807',
 'customfield_100

In [6]:
field_mapping

[{'clauseNames': ['issuetype', 'type'],
  'custom': False,
  'id': 'issuetype',
  'name': 'Issue Type',
  'navigable': True,
  'orderable': True,
  'schema': {'system': 'issuetype', 'type': 'issuetype'},
  'searchable': True},
 {'clauseNames': ['timespent'],
  'custom': False,
  'id': 'timespent',
  'name': 'Time Spent',
  'navigable': True,
  'orderable': False,
  'schema': {'system': 'timespent', 'type': 'number'},
  'searchable': False},
 {'clauseNames': ['project'],
  'custom': False,
  'id': 'project',
  'name': 'Project',
  'navigable': True,
  'orderable': False,
  'schema': {'system': 'project', 'type': 'project'},
  'searchable': True},
 {'clauseNames': ['cf[15002]', 'RF Client ID'],
  'custom': True,
  'id': 'customfield_15002',
  'name': 'RF Client ID',
  'navigable': True,
  'orderable': True,
  'schema': {'custom': 'com.atlassian.jira.plugin.system.customfieldtypes:textfield',
   'customId': 15002,
   'type': 'string'},
  'searchable': True},
 {'clauseNames': ['fixVersion'

In [7]:
explode_issues

,σ_progress.percent,σ_progress.progress,σ_progress.total,aggregateremaining_estimate,σ_original_estimate,aggregatetime_spent,assignee,assignee.active,assignee.avatarurls.16x16,assignee.avatarurls.24x24,...,time_spent,updated,affects_version/s,votes.hasvoted,votes.self,votes.votes,watchers.iswatching,watchers.self,watchers.watchcount,work_ratio
0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:55:33.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1215/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1215/w...,1,-1
1,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:55:33.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1214/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1214/w...,1,-1
2,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:54:58.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1213/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1213/w...,1,-1
3,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:54:56.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1212/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1212/w...,1,-1
4,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:54:55.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1211/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1211/w...,1,-1
5,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:54:53.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1210/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1210/w...,1,-1
6,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:54:52.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1209/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1209/w...,1,-1
7,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:54:51.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1208/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1208/w...,1,-1
8,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:54:49.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1207/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1207/w...,1,-1
9,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-11-16T22:54:48.000+0000,[],False,http://10.221.100.4/rest/api/2/issue/AI-1206/v...,0,True,http://10.221.100.4/rest/api/2/issue/AI-1206/w...,1,-1


In [8]:
explode_issues.columns.tolist()

['σ_progress.percent',
 'σ_progress.progress',
 'σ_progress.total',
 'aggregateremaining_estimate',
 'σ_original_estimate',
 'aggregatetime_spent',
 'assignee',
 'assignee.active',
 'assignee.avatarurls.16x16',
 'assignee.avatarurls.24x24',
 'assignee.avatarurls.32x32',
 'assignee.avatarurls.48x48',
 'assignee.displayname',
 'assignee.emailaddress',
 'assignee.key',
 'assignee.name',
 'assignee.self',
 'assignee.timezone',
 'component/s',
 'created',
 'creator.active',
 'creator.avatarurls.16x16',
 'creator.avatarurls.24x24',
 'creator.avatarurls.32x32',
 'creator.avatarurls.48x48',
 'creator.displayname',
 'creator.emailaddress',
 'creator.key',
 'creator.name',
 'creator.self',
 'creator.timezone',
 'sprint',
 'epic_link',
 'rank_(obsolete)',
 'epic_status.id',
 'epic_status.self',
 'epic_status.value',
 'epic_name',
 'epic_color',
 'flagged',
 'epic/theme',
 'story_points',
 'business_value',
 'release_version_history',
 'external_reference',
 'desired_key',
 'affected_user_groups',